### Creates `openalex.works.authors_and_affiliations` in Walden End to End workflow

In [0]:
-- Step 1a: Create intermediate table with best authors (optimized, no complex joins yet)
-- Ensure final table exists
CREATE TABLE IF NOT EXISTS identifier('openalex' || :env_suffix || '.works.authors_and_affiliations') (
  work_id BIGINT,
  updated_datetime TIMESTAMP,
  authorships ARRAY<STRUCT<
    affiliations: ARRAY<STRUCT<institution_ids: ARRAY<STRING>, raw_affiliation_string: STRING>>,
    author: STRUCT<display_name: STRING, id: STRING, orcid: STRING>,
    author_position: STRING,
    author_order_number: INT,
    countries: ARRAY<STRING>,
    institutions: ARRAY<STRUCT<country_code: STRING, display_name: STRING, id: STRING, lineage: ARRAY<STRING>, ror: STRING, type: STRING, type_list: ARRAY<STRING>>>,
    is_corresponding: BOOLEAN,
    raw_affiliation_strings: ARRAY<STRING>,
    raw_author_name: STRING
  >>
) USING DELTA CLUSTER BY (work_id);

DECLARE OR REPLACE VARIABLE max_updated_date TIMESTAMP DEFAULT to_timestamp('1900-01-01');
SET VARIABLE max_updated_date = COALESCE((SELECT MAX(updated_datetime) FROM identifier('openalex' || :env_suffix || '.works.authors_and_affiliations')), to_timestamp('1900-01-01'));

-- Step 1a: Create intermediate table with filtered, deduplicated works, exploded authors with metadata
CREATE OR REPLACE TABLE identifier('openalex' || :env_suffix || '.works.authors_and_affiliations_stage1')
USING DELTA
CLUSTER BY (work_id)
TBLPROPERTIES (
    'delta.autoOptimize.optimizeWrite' = 'true',
    'delta.autoOptimize.autoCompact' = 'true'
) AS
WITH deduplicated_works AS (
    SELECT
        work_id,
        authors,
        openalex_updated_dt AS updated_datetime,
        get(authors.affiliations.name, 0) IS NOT NULL AS affiliations_exist,
        EXISTS(authors.is_corresponding, x -> x = TRUE) AS is_corresponding_exists,
        ROW_NUMBER() OVER (
            PARTITION BY work_id
            ORDER BY priority ASC
        ) AS r
    FROM identifier('openalex' || :env_suffix || '.works.locations_mapped')
    LEFT JOIN openalex.system.priority_table USING (provenance)
    WHERE authors_exist
      AND openalex_updated_dt >= max_updated_date
),
best_works AS (
    SELECT
        work_id,
        updated_datetime,
        authors,
        affiliations_exist,
        is_corresponding_exists,
        array_size(authors) AS author_count
    FROM deduplicated_works
    WHERE r = 1
),
exploded_authors AS (
    SELECT
        work_id,
        updated_datetime,
        author_count,
        affiliations_exist,
        is_corresponding_exists,
        posexplode(authors) AS (original_author_order, author_data)
    FROM best_works
),
with_corresponding_flag AS (
    SELECT
        work_id,
        MAX(CASE WHEN author_data.is_corresponding = TRUE THEN 1 ELSE 0 END) AS has_corresponding
    FROM exploded_authors
    WHERE is_corresponding_exists
    GROUP BY work_id
)
SELECT
    ea.work_id,
    ea.updated_datetime,
    ea.original_author_order,
    ea.author_count,
    TRIM(ea.author_data.name) AS raw_author_name,
    ea.author_data.affiliations.name AS raw_affiliation_strings,
    ea.author_data.is_corresponding AS is_corresponding_from_source,
    COALESCE(wc.has_corresponding, 0) AS work_has_corresponding,
    -- Join author metadata
    a.display_name,
    a.author_id,
    ao.orcid
FROM exploded_authors ea
LEFT JOIN with_corresponding_flag wc ON ea.work_id = wc.work_id
LEFT JOIN openalex.mid.author a
  ON openalex.works.normalize_affiliation_string(TRIM(ea.author_data.name)) = a.normalized_name
LEFT JOIN openalex.mid.author_orcid ao ON a.author_id = ao.author_id;

In [ ]:
-- Step 1b: Build raw authorships from stage1 (already has author metadata joined)
CREATE OR REPLACE TABLE identifier('openalex' || :env_suffix || '.works.authors_and_affiliations_updates')
CLUSTER BY (work_id) AS (
WITH merged_duplicates AS (
    SELECT
        work_id,
        updated_datetime,
        original_author_order,
        author_count,
        MIN_BY(raw_author_name, original_author_order) AS raw_author_name,
        MAX_BY(is_corresponding_from_source, original_author_order) AS is_corresponding_from_source,
        FIRST(work_has_corresponding, true) AS work_has_corresponding,
        FIRST(display_name, true) AS display_name,
        FIRST(author_id, true) AS author_id,
        FIRST(orcid, true) AS orcid,
        ARRAY_DISTINCT(ARRAY_COMPACT(FLATTEN(COLLECT_LIST(raw_affiliation_strings)))) AS raw_affiliation_strings
    FROM identifier('openalex' || :env_suffix || '.works.authors_and_affiliations_stage1')
    GROUP BY work_id, updated_datetime, original_author_order, author_count
)
SELECT
    work_id,
    updated_datetime,
    TRANSFORM(
        ARRAY_SORT(
            COLLECT_LIST(
                STRUCT(
                    original_author_order,
                    STRUCT(
                        CASE
                            WHEN original_author_order == 0 THEN 'first'
                            WHEN original_author_order + 1 == author_count THEN 'last'
                            ELSE 'middle'
                        END AS author_position,
                        TRIM(REPLACE(raw_author_name, '\n', '')) AS raw_author_name,
                        CASE
                            WHEN is_corresponding_from_source = TRUE THEN TRUE
                            WHEN work_has_corresponding = 1 THEN FALSE
                            WHEN original_author_order == 0 THEN TRUE
                            ELSE FALSE
                        END AS is_corresponding,
                        TRANSFORM(raw_affiliation_strings, s -> TRIM(REPLACE(s, '\n', ''))) AS raw_affiliation_strings,
                        original_author_order,
                        NAMED_STRUCT(
                            'display_name', COALESCE(display_name, TRIM(REPLACE(raw_author_name, '\n', ''))),
                            'id', CASE WHEN author_id IS NOT NULL THEN CONCAT('https://openalex.org/A', author_id) ELSE NULL END,
                            'orcid', CASE WHEN orcid IS NOT NULL THEN CONCAT('https://orcid.org/', orcid) ELSE NULL END
                        ) AS author
                    ) AS authorship
                )
            ),
            (left, right) -> CASE
                WHEN left.original_author_order < right.original_author_order THEN -1
                WHEN left.original_author_order > right.original_author_order THEN 1
                ELSE 0
            END
        ),
        x -> x.authorship
    ) AS authorships
FROM merged_duplicates
GROUP BY work_id, updated_datetime
);


## Merge new affiliation strings into `affiliation_strings_lookup` table

In [0]:
-- Step 2: Merge new raw affiliation strings into lookup table
MERGE INTO openalex.institutions.affiliation_strings_lookup AS target
USING (
    WITH new_affiliation_strings AS (
        SELECT DISTINCT
            affiliation_string AS raw_affiliation_string
        FROM identifier('openalex' || :env_suffix || '.works.authors_and_affiliations_updates')
        LATERAL VIEW explode(authorships) AS authorship
        LATERAL VIEW explode(authorship.raw_affiliation_strings) AS affiliation_string
        WHERE affiliation_string IS NOT NULL 
          AND affiliation_string != ""
    )
    SELECT 
        nas.raw_affiliation_string,
        CAST(NULL AS ARRAY<BIGINT>) AS institution_ids,
        CAST(NULL AS ARRAY<BIGINT>) AS institution_ids_override,
        CURRENT_TIMESTAMP() AS created_datetime
    FROM new_affiliation_strings nas
    LEFT JOIN openalex.institutions.affiliation_strings_lookup existing
        ON nas.raw_affiliation_string = existing.raw_affiliation_string
    WHERE existing.raw_affiliation_string IS NULL
) AS source
ON target.raw_affiliation_string = source.raw_affiliation_string
WHEN NOT MATCHED THEN
    INSERT (raw_affiliation_string, institution_ids, institution_ids_override, created_datetime)
    VALUES (source.raw_affiliation_string, source.institution_ids, source.institution_ids_override, source.created_datetime);

### Create and populate `openalex.authors.author_institutions` mapping

In [0]:
-- Step 3: author_institutions mapping
-- CREATE TABLE IF NOT EXISTS identifier('openalex' || :env_suffix || '.authors.author_institutions')
CREATE OR REPLACE TABLE identifier('openalex' || :env_suffix || '.authors.author_institutions')  -- replace every time for now
USING DELTA
CLUSTER BY (work_id)
TBLPROPERTIES (
    'delta.autoOptimize.optimizeWrite' = 'true',
    'delta.autoOptimize.autoCompact' = 'true'
) AS
SELECT 
    CAST(NULL AS BIGINT) AS work_id,
    CAST(NULL AS BIGINT) AS author_sequence,
    CAST(NULL AS BIGINT) AS institution_id,
    CAST(NULL AS STRING) AS raw_author_name,
    CAST(NULL AS STRING) AS raw_affiliation_string,
    CAST(NULL as ARRAY<STRING>) as raw_countries
WHERE 1=0;

-- Execute merge
MERGE INTO identifier('openalex' || :env_suffix || '.authors.author_institutions') AS target
USING (
    WITH new_works AS (
        SELECT DISTINCT work_id
        FROM identifier('openalex' || :env_suffix || '.works.authors_and_affiliations_updates') aa
        -- WHERE NOT EXISTS (
        --     SELECT 1 
        --     FROM identifier('openalex' || :env_suffix || '.authors.author_institutions') ai
        --     WHERE ai.work_id = aa.work_id
        -- )
    ),
    exploded_authorships AS (
        SELECT 
            work_id,
            posexplode(authorships) AS (author_sequence, authorship),
            authorship.original_author_order AS original_author_order
        FROM identifier('openalex' || :env_suffix || '.works.authors_and_affiliations_updates')
        WHERE work_id IN (SELECT work_id FROM new_works)
    ),
    
    exploded_affiliations AS (
        SELECT 
            work_id,
            author_sequence,
            original_author_order,
            authorship.raw_author_name,
            authorship.author_position,
            authorship.is_corresponding,
            posexplode_outer(authorship.raw_affiliation_strings) AS (affiliation_sequence, raw_affiliation_string)
        FROM exploded_authorships
    ),
    
    with_institutions AS (
        SELECT 
            ea.work_id,
            ea.author_sequence,
            ea.raw_author_name,
            ea.raw_affiliation_string,
            asl.countries as raw_countries,
            CASE 
                WHEN ea.raw_affiliation_string IS NULL THEN NULL
                WHEN asl.institution_ids_override IS NOT NULL AND SIZE(asl.institution_ids_override) > 0 
                    THEN asl.institution_ids_override
                WHEN asl.institution_ids IS NOT NULL AND SIZE(asl.institution_ids) > 0 
                    AND NOT (SIZE(asl.institution_ids) = 1 AND asl.institution_ids[0] = -1) 
                    THEN asl.institution_ids
                ELSE NULL
            END AS institution_ids
        FROM exploded_affiliations ea
        LEFT JOIN openalex.institutions.affiliation_strings_lookup asl
            ON ea.raw_affiliation_string = asl.raw_affiliation_string
            AND ea.raw_affiliation_string IS NOT NULL
    )
    
    SELECT 
        work_id,
        author_sequence,
        institution_id,
        raw_author_name,
        raw_affiliation_string,
        raw_countries
    FROM with_institutions
    LATERAL VIEW OUTER explode(institution_ids) AS institution_id
    WHERE institution_id IS NOT NULL
) AS source
ON target.work_id = source.work_id 
   AND target.author_sequence = source.author_sequence 
   AND NVL(target.institution_id, -1) = NVL(source.institution_id, -1)
   AND NVL(target.raw_affiliation_string, '') = NVL(source.raw_affiliation_string, '')
WHEN NOT MATCHED THEN
    INSERT (work_id, author_sequence, institution_id, raw_author_name, raw_affiliation_string, raw_countries)
    VALUES (source.work_id, source.author_sequence, source.institution_id, 
            source.raw_author_name, source.raw_affiliation_string, source.raw_countries);

### Add institutions arrays to authorships

In [0]:
%run ../utils/variables

In [0]:
-- Step 4: Enrich updates with full institution details to complete the schema
CREATE OR REPLACE TABLE identifier('openalex' || :env_suffix || '.works.authors_and_affiliations_updates')
CLUSTER BY (work_id) AS (
WITH existing_authorships AS (
    SELECT
        work_id,
        updated_datetime,
        posexplode(authorships) AS (author_sequence, authorship),
        authorship.original_author_order AS original_author_order
    FROM identifier('openalex' || :env_suffix || '.works.authors_and_affiliations_updates')
),
institution_lineage AS (
  SELECT
    institution_id,
    COLLECT_LIST(ancestor_id) AS lineage_ids
  FROM openalex.mid.institution_ancestors
  WHERE NOT ARRAY_CONTAINS(SUPER_SYSTEM_INSTITUTIONS, ancestor_id)
  GROUP BY institution_id
),
author_institutions_with_details AS (
    SELECT
        ai.work_id,
        ai.author_sequence,
        array_distinct(flatten(collect_set(ai.raw_countries))) as raw_parsed_countries,
        COLLECT_SET(
            STRUCT(
                inst.iso3166_code as country_code,
                inst.display_name,
                CONCAT('https://openalex.org/I', ai.institution_id) AS id,
                ARRAY_SORT(
                    TRANSFORM(
                    ARRAY_COMPACT(CONCAT(ARRAY(ai.institution_id), COALESCE(il.lineage_ids, ARRAY()))), id -> CONCAT('https://openalex.org/I', id)
                    )
                ) AS lineage,
                CONCAT('https://ror.org/', inst.ror_id) AS ror,
                inst.type,
                CAST(ARRAY(inst.type) as array<string>) as type_list
            )
        ) AS institutions
    FROM identifier('openalex' || :env_suffix || '.authors.author_institutions') ai
    LEFT JOIN openalex.institutions.institutions inst ON inst.id = ai.institution_id
    LEFT JOIN institution_lineage il USING (institution_id)
    WHERE ai.institution_id IS NOT NULL
    GROUP BY ai.work_id, ai.author_sequence
),
-- Map raw_affiliation_string -> institution_ids per (work_id, author_sequence)
affiliations_map_ids AS (
    SELECT
        work_id,
        author_sequence,
        raw_affiliation_string,
        array_distinct(
            array_compact(
                COLLECT_LIST(CONCAT('https://openalex.org/I', institution_id))
            )
        ) AS institution_ids
    FROM identifier('openalex' || :env_suffix || '.authors.author_institutions')
    WHERE institution_id IS NOT NULL
      AND raw_affiliation_string IS NOT NULL
    GROUP BY work_id, author_sequence, raw_affiliation_string
),
affiliations_map AS (
    SELECT
        work_id,
        author_sequence,
        MAP_FROM_ENTRIES(
            COLLECT_LIST(NAMED_STRUCT('key', raw_affiliation_string, 'value', institution_ids))
        ) AS aff_map
    FROM affiliations_map_ids
    GROUP BY work_id, author_sequence
),
authorships_with_institutions AS (
    SELECT
        ea.work_id,
        ea.author_sequence,
        ea.original_author_order,
        -- Rebuilding the authorship struct to strictly match the target schema
        STRUCT(
            -- 1. 'affiliations' is built from raw_strings, populate institution_ids from author_institutions per string
            TRANSFORM(
                COALESCE(ea.authorship.raw_affiliation_strings, ARRAY()),
                s -> STRUCT(
                    COALESCE(element_at(am.aff_map, s), ARRAY()) as institution_ids,
                    s as raw_affiliation_string
                )
            ) as affiliations,
            -- 2. 'author' propagated from raw updates
            COALESCE(ea.authorship.author, CAST(NULL AS STRUCT<display_name: STRING, id: STRING, orcid: STRING>)) AS author,
            -- 3. All other fields are populated from the available data
            ea.authorship.author_position,
            ea.original_author_order AS author_order_number,
            CASE
                WHEN size(FILTER(aid.institutions.country_code, c -> c IS NOT NULL AND c <> '')) > 0
                    THEN array_sort(array_distinct(FILTER(aid.institutions.country_code, c -> c IS NOT NULL AND c <> '')))
                WHEN aid.raw_parsed_countries IS NOT NULL
                    AND aid.raw_parsed_countries <> ARRAY("")
                THEN aid.raw_parsed_countries
                ELSE ARRAY()
            END AS countries,
            COALESCE(aid.institutions, ARRAY()) AS institutions,
            ea.authorship.is_corresponding,
            ea.authorship.raw_affiliation_strings,
            ea.authorship.raw_author_name
        ) AS authorship_with_institutions
    FROM existing_authorships ea
    LEFT JOIN author_institutions_with_details aid
        ON ea.work_id = aid.work_id
        AND ea.author_sequence = aid.author_sequence
    LEFT JOIN affiliations_map am
        ON ea.work_id = am.work_id
        AND ea.author_sequence = am.author_sequence
)
SELECT
    work_id,
    updated_datetime,
    TRANSFORM(
        ARRAY_SORT(
            COLLECT_LIST(
                STRUCT(
                    original_author_order,
                    authorship_with_institutions
                )
            ),
            (left, right) -> CASE
                WHEN left.original_author_order < right.original_author_order THEN -1
                WHEN left.original_author_order > right.original_author_order THEN 1
                ELSE 0
            END
        ),
        x -> x.authorship_with_institutions
    ) AS authorships
FROM authorships_with_institutions
GROUP BY work_id, updated_datetime
);

In [ ]:
-- Step 5: Merge enriched updates into final authors_and_affiliations table
MERGE INTO identifier('openalex' || :env_suffix || '.works.authors_and_affiliations') AS target
USING identifier('openalex' || :env_suffix || '.works.authors_and_affiliations_updates') AS source
ON target.work_id = source.work_id
WHEN MATCHED THEN UPDATE SET
  target.authorships = source.authorships,
  target.updated_datetime = source.updated_datetime
WHEN NOT MATCHED THEN INSERT (work_id, authorships, updated_datetime)
VALUES (source.work_id, source.authorships, source.updated_datetime);
